In [1]:
import sys
import os

# Caminho do diretório pai em relação ao diretório atual
parent_dir = os.path.abspath(os.path.join(os.getcwd(), ".."))

# Adiciona ao sys.path
sys.path.append(parent_dir)

from graphgen_models.UnAtt.func import *
import torch
import torch_geometric.datasets as datasets
from utils.utils import count_hetero_edges, split_edge_index_by_label, get_data

from graphgen_models.UnAtt.model import *

from torch_geometric.datasets import Planetoid

data_to_mimic = Planetoid(root = '../datasets', name = 'Cora')[0]

In [4]:
# n = 2 * torch.unique(data_to_mimic.y, return_counts=True)[1]
n = 0

g = unatt(data_to_mimic=data_to_mimic, number_of_nodes=n)

g.edge_generation()

[UnAtt] Init (mimic): k=7, sizes=[351, 217, 418, 818, 426, 298, 180]
534 edges to be added to cluster 0

409 edges to be added to cluster 1

827 edges to be added to cluster 2

1175 edges to be added to cluster 3

660 edges to be added to cluster 4

417 edges to be added to cluster 5

253 edges to be added to cluster 6



In [6]:
# (tensor([0, 1, 2, 3, 4, 5, 6]), tensor([351, 217, 418, 818, 426, 298, 180]))
import networkx as nx
for label in range(7):
    G = nx.read_gml(f'../datasets/GenCAT/Planetoid:Cora_n_2_m_2/gmls/Planetoid:Cora_{label}.gml')

    print(f'original edges for label {label}', G.number_of_edges())
    print(f'original nodes for label {label}', G.number_of_nodes())

    G = nx.read_gml(f'../datasets/Cora/gmls/Planetoid:Cora_{label}.gml')

    # print(f'generated edges for label {label}', len(split_edge_index_by_label(g.whole_graph_edge_index, g.y)[label][0]))
    # print(f'generated nodes for label {label}', g.number_of_nodes[label].item())
    print(f'original edges for label {label}', G.number_of_edges())
    print(f'original nodes for label {label}', G.number_of_nodes())

original edges for label 0 1790
original nodes for label 0 748
original edges for label 0 534
original nodes for label 0 329
original edges for label 1 1664
original nodes for label 1 464
original edges for label 1 409
original nodes for label 1 196
original edges for label 2 3261
original nodes for label 2 881
original edges for label 2 827
original nodes for label 2 410
original edges for label 3 5006
original nodes for label 3 1547
original edges for label 3 1175
original nodes for label 3 767
original edges for label 4 2422
original nodes for label 4 820
original edges for label 4 660
original nodes for label 4 400
original edges for label 5 1817
original nodes for label 5 667
original edges for label 5 417
original nodes for label 5 279
original edges for label 6 886
original nodes for label 6 289
original edges for label 6 253
original nodes for label 6 170


In [6]:
g.number_of_nodes

tensor([351, 217, 418, 818, 426, 298, 180])

In [7]:
torch.unique(data_to_mimic.y, return_counts=True)

(tensor([0, 1, 2, 3, 4, 5, 6]), tensor([351, 217, 418, 818, 426, 298, 180]))

In [8]:
len(split_edge_index_by_label(g.whole_graph_edge_index, g.y)[6][0])

506

In [9]:
torch.unique(g.pdf_distributions[6])

tensor([0.0015, 0.0029, 0.0044, 0.0058, 0.0073, 0.0087, 0.0102, 0.0117, 0.0131,
        0.0219, 0.0408])

In [10]:
len(g.pdf_distributions[0])

351

In [11]:
g.nodes[1][0]

tensor(351, dtype=torch.int32)

In [12]:
sum(g.noise_class_distribution)

tensor(1.)

# Testing output analysis

In [59]:
import pandas as pd
import os

import warnings

warnings.warn('ignore')

cols = ["Strongest", "Strong", "Weak", "Weakest", "Super_Weak"]

def marcar_sem_evidencia(df, cols, nova_coluna="sem_evidencia"):
    """
    Adiciona uma nova coluna ao DataFrame indicando se todas as colunas
    em `cols` são zero para cada linha.
    
    Parâmetros:
    - df: DataFrame de entrada.
    - cols: Lista de colunas para verificar.
    - nova_coluna: Nome da nova coluna a ser adicionada.
    
    Retorna:
    - Uma cópia do DataFrame com a nova coluna.
    """
    df = df.copy()
    df[nova_coluna] = df[cols].sum(axis=1) == 0
    return df

# def category_binary(category):
#     cat2bin = {"Strongest" : 5,
#                "Strong": 4,
#                "Weak": 3,
#                "Weakest": 2,
#                "Super_Weak": 1}
    
#     return cat2bin[category]



model = "GenCAT"
dataset = "Planetoid:Cora"
n = 2
m = 2

# Caminho do diretório que você quer verificar
folder = f'../outputs/{model}/{dataset}/'

# Prefixo que os arquivos devem ter
prefix = f'network_category_n_{n}_m_{m}'

# Listar todos os arquivos que começam com o prefixo
df_paths = [f for f in os.listdir(folder) if f.startswith(prefix) and os.path.isfile(os.path.join(folder, f))]

/tmp/ipykernel_145349/1263536584.py:6: UserWarning: ignore
  warnings.warn('ignore')


In [60]:
dfs = list()

df_reference = pd.read_csv(f'../outputs/{dataset}/network_category_n_0_m_0.csv')[['Unnamed: 0'] + cols]

for fn in df_paths:
    df = pd.read_csv(folder + fn)[['Unnamed: 0'] + cols]
    df = marcar_sem_evidencia(df, cols = cols)
    df.iloc[:,1:] = df.iloc[:,1:].astype(int)

    dfs.append(df)

/tmp/ipykernel_145349/3648192058.py:8: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '0    0
1    0
2    0
3    0
4    0
5    0
6    1
7    0
Name: Strongest, dtype: int64' has dtype incompatible with bool, please explicitly cast to a compatible dtype first.
  df.iloc[:,1:] = df.iloc[:,1:].astype(int)
/tmp/ipykernel_145349/3648192058.py:8: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '0    0
1    0
2    0
3    0
4    0
5    0
6    1
7    0
Name: Strong, dtype: int64' has dtype incompatible with bool, please explicitly cast to a compatible dtype first.
  df.iloc[:,1:] = df.iloc[:,1:].astype(int)
/tmp/ipykernel_145349/3648192058.py:8: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '0    1
1    1
2    1
3    1
4    1
5    1
6    1
7    1
Name: Weak, dtype: int64' has dtype incomp

In [61]:
dfs

[                                          Unnamed: 0  Strongest  Strong  Weak  \
 0  datasets/GenCAT/Planetoid:Cora_n_2_m_2/2025-09...          0       0     1   
 1  datasets/GenCAT/Planetoid:Cora_n_2_m_2/2025-09...          0       0     1   
 2  datasets/GenCAT/Planetoid:Cora_n_2_m_2/2025-09...          0       0     1   
 3  datasets/GenCAT/Planetoid:Cora_n_2_m_2/2025-09...          0       0     1   
 4  datasets/GenCAT/Planetoid:Cora_n_2_m_2/2025-09...          0       0     1   
 5  datasets/GenCAT/Planetoid:Cora_n_2_m_2/2025-09...          0       0     1   
 6  datasets/GenCAT/Planetoid:Cora_n_2_m_2/2025-09...          1       1     1   
 7  datasets/GenCAT/Planetoid:Cora_n_2_m_2/2025-09...          0       0     1   
 
    Weakest  Super_Weak  sem_evidencia  
 0        1           0              0  
 1        1           0              0  
 2        1           0              0  
 3        1           0              0  
 4        1           0              0  
 5        1   

In [62]:
dfs_num = [df.drop(columns=["Unnamed: 0"]) for df in dfs]

In [63]:
dfs_num

[   Strongest  Strong  Weak  Weakest  Super_Weak  sem_evidencia
 0          0       0     1        1           0              0
 1          0       0     1        1           0              0
 2          0       0     1        1           0              0
 3          0       0     1        1           0              0
 4          0       0     1        1           0              0
 5          0       0     1        1           0              0
 6          1       1     1        1           1              0
 7          0       0     1        1           0              0,
    Strongest  Strong  Weak  Weakest  Super_Weak  sem_evidencia
 0          0       0     1        1           0              0
 1          0       0     1        1           0              0
 2          0       0     1        1           0              0
 3          0       0     1        1           0              0
 4          1       1     1        1           1              0
 5          0       0     1        1   

In [64]:
df_soma = sum(dfs_num)

In [65]:
df_soma

,Strongest,Strong,Weak,Weakest,Super_Weak,sem_evidencia
0,0,0,4,4,0,1
1,0,0,5,5,0,0
2,0,0,4,4,0,1
3,0,0,5,5,0,0
4,1,1,5,5,1,0
5,0,0,4,4,1,1
6,1,1,5,5,4,0
7,0,0,2,2,0,3
